In [1]:
import os

import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from utils.constants import base_path, label_dict
from utils.parsbert import train_parsbert_with_l2, test_parsbert, predict_parsbert
from utils.preprocess import clean_text

# Load raw datasets

In [2]:
pd.set_option('future.no_silent_downcasting', True)  # noqa

# Load datasets
# tsv: uses tab as delimiter instead of comma in csv
train_df = pd.read_csv(f"{base_path}/data/train.tsv", sep="\t", header=None, names=["sentence", "label"])
test_df = pd.read_csv(f"{base_path}/data/test.tsv", sep="\t", header=None, names=["sentence", "label"])

# print distinct labels
print(train_df.label.unique())
print(test_df.label.unique())
print(set(train_df.label.unique()) == set(test_df.label.unique()))
print('*' * 50)

print(label_dict)
print(train_df.loc[0, "sentence"])
print(train_df.loc[0, "label"])
print('*' * 50)

print(train_df.dtypes)

['SAD' 'HATE' 'OTHER' 'FEAR' 'ANGRY' 'HAPPY' 'SURPRISE']
['SAD' 'HAPPY' 'OTHER' 'SURPRISE' 'FEAR' 'HATE' 'ANGRY']
True
**************************************************
{'HAPPY': 0, 'SAD': 1, 'ANGRY': 2, 'FEAR': 3, 'SURPRISE': 4, 'HATE': 5, 'OTHER': 6}
خیلی کوچیک هستن و سایزشون بدرد نمیخوره میخوام پس بدم
SAD
**************************************************
sentence    object
label       object
dtype: object


## clean data if it's not already done

In [3]:
# clean data and save
tqdm.pandas()
if not os.path.exists(f"{base_path}/data/train_cleaned.tsv"):
    train_df['sentence'] = train_df['sentence'].progress_apply(clean_text)
    train_df.to_csv(f"{base_path}/data/train_cleaned.tsv", sep="\t", index=False)
if not os.path.exists(f"{base_path}/data/test_cleaned.tsv"):
    test_df['sentence'] = test_df['sentence'].progress_apply(clean_text)
    test_df.to_csv(f"{base_path}/data/test_cleaned.tsv", sep="\t", index=False)

# Function to apply preprocessing in parallel using joblib
# def parallel_apply(df, func):
#     processed_sentences = Parallel(n_jobs=-1)(
#         delayed(func)(text=sentence) for sentence in df['sentence'])
#     return processed_sentences


# if not os.path.exists(f"{base_path}/data/train_cleaned.tsv"):
#     train_df['sentence'] = parallel_apply(train_df, combined_preprocess)

# if not os.path.exists(f"{base_path}/data/test_cleaned.tsv"):
#     test_df['sentence'] = parallel_apply(test_df, combined_preprocess)

# Load cleaned datasets

In [4]:
# First row is header
train_df = pd.read_csv(f"{base_path}/data/train_cleaned.tsv", sep="\t")
test_df = pd.read_csv(f"{base_path}/data/test_cleaned.tsv", sep="\t")

print(train_df.loc[0:5, "sentence"])
print(test_df.loc[0:5, "sentence"])
print(test_df.loc[6, "sentence"])

0    خیلی کوچیک هستن و سایزشون بدرد نمیخوره میخوام ...
1       از صدای پرنده دم دمای صبح متنفرم متنفرم متنفرم
2    کیفیتش خیلی خوبه با شک خریدم ولی واقعا راضیم ب...
3    چون همش با دوربین ثبت‌شده ایا میشه اعتراض زد؟؟...
4                    این وضع ب طرز خنده داری گریه داره
5    خب من رسما از یک نفر متنفرم چون از گربه بدش می...
Name: sentence, dtype: object
0    این شاید اولین عزای عمومی واقعی است که یاد دار...
1    دیشب بعد از ارسال تویت مربوط به آثار باستانی ت...
2    کدوم شعبه پول نداده بگو الان برات آمار دقیق بد...
3    امروز وسط یه بحث با بابا مامانم گفتم آدم باید ...
4    امشب گفت نامزدی دوستش که ادم روشنفکری است بهم ...
5    به امید موفقیت تیم ملی و پیروزی در بازی امروز ...
Name: sentence, dtype: object
با آرزوی موفقیت و پیروزی


In [5]:
# Split train dataset for validation
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df['sentence'], train_df['label'], test_size=0.1, random_state=42
)
print(f"Number of training sentences: {len(train_sentences)}")
print(f"Number of validation sentences: {len(val_sentences)}")
print(f"Number of test sentences: {len(test_df)}")

Number of training sentences: 5512
Number of validation sentences: 613
Number of test sentences: 1151


In [ ]:
# write each label count per dataset
print(f"Train counts: {train_labels.value_counts().to_dict()}")
print(f"Val counts: {val_labels.value_counts().to_dict()}")
print(f"Test counts: {test_df['label'].value_counts().to_dict()}")
# highly imbalanced dataset

Train counts: {'OTHER': 1492, 'ANGRY': 819, 'SAD': 806, 'FEAR': 691, 'SURPRISE': 677, 'HAPPY': 564, 'HATE': 463}
Val counts: {'OTHER': 189, 'ANGRY': 104, 'SAD': 90, 'FEAR': 66, 'SURPRISE': 62, 'HAPPY': 54, 'HATE': 48}
Test counts: {'HAPPY': 275, 'SAD': 262, 'OTHER': 193, 'ANGRY': 154, 'SURPRISE': 145, 'HATE': 65, 'FEAR': 57}


# Load the tokenizer and model

In [7]:
model_names = [
    'HooshvareLab/bert-fa-base-uncased-sentiment-snappfood',  # noqa
    'HooshvareLab/bert-fa-base-uncased-sentiment-digikala',  # noqa
    'HooshvareLab/bert-fa-base-uncased',  # noqa
    'HooshvareLab/bert-fa-zwnj-base'  # ParsBERT (v3.0) # noqa (newest: chat GPT suggested)
]
cache_dir = f'{base_path}/models/huggingface_cache' # where to save models

# Set up the training arguments

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

for model_name in model_names:
    print(f"Model name: {model_name}")
    train_parsbert_with_l2(
        model_name=model_name,
        cache_dir=cache_dir,
        device=device,
        label_dict=label_dict,
        train_sentences=train_sentences,
        train_labels=train_labels,
        val_sentences=val_sentences,
        val_labels=val_labels,
        base_path=base_path
    )

Device: cuda
Model name: HooshvareLab/bert-fa-base-uncased-sentiment-snappfood


/home/hamedhf/anaconda3/envs/DL-project/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased-sentiment-snappfood and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training Epoch 1: 100%|█

Epoch 1: Train Loss: 1.1846 | Train Accuracy: 0.5677


Validation Epoch 1: 100%|██████████| 77/77 [00:10<00:00,  7.38it/s]


Epoch 1: Val Loss: 0.8939 | Val Accuracy: 0.6884
**************************************************
Epoch 1: New best model saved with val_loss 0.8939 & val_acc 0.6884


...

Epoch 11: Val Loss: 1.8476 | Val Accuracy: 0.6639
**************************************************


Training Epoch 12: 100%|██████████| 689/689 [04:25<00:00,  2.60it/s]


Epoch 12: Train Loss: 0.0067 | Train Accuracy: 0.9980


Validation Epoch 12: 100%|██████████| 77/77 [00:09<00:00,  8.03it/s]

Epoch 12: Val Loss: 1.8859 | Val Accuracy: 0.6591
**************************************************


In [9]:
for model_name in model_names:
    print(f"Model name: {model_name}")
    test_parsbert(
        model_name=model_name,
        cache_dir=cache_dir,
        device=device,
        label_dict=label_dict,
        test_sentences=test_df['sentence'].to_list(),
        test_labels=test_df['label'].to_list(),
        base_path=base_path
    )

Model name: HooshvareLab/bert-fa-base-uncased-sentiment-snappfood


Testing: 100%|██████████| 144/144 [00:19<00:00,  7.27it/s]


Test Loss: 1.1277 | Test Accuracy: 0.6195
Precision: 0.6543 | Recall: 0.6082 | F1 Score: 0.6094
**************************************************
Model name: HooshvareLab/bert-fa-base-uncased-sentiment-digikala


Testing: 100%|██████████| 144/144 [00:18<00:00,  7.92it/s]


Test Loss: 1.1970 | Test Accuracy: 0.6299
Precision: 0.6829 | Recall: 0.6037 | F1 Score: 0.6164
**************************************************
Model name: HooshvareLab/bert-fa-base-uncased


Testing: 100%|██████████| 144/144 [00:18<00:00,  7.93it/s]


Test Loss: 1.1620 | Test Accuracy: 0.5821
Precision: 0.6493 | Recall: 0.5669 | F1 Score: 0.5763
**************************************************
Model name: HooshvareLab/bert-fa-zwnj-base


Testing: 100%|██████████| 144/144 [00:18<00:00,  7.78it/s]

Test Loss: 1.1727 | Test Accuracy: 0.5960
Precision: 0.6357 | Recall: 0.5945 | F1 Score: 0.5976
**************************************************


In [10]:
# Example usage
model_name = "HooshvareLab/bert-fa-base-uncased" # can change the model name on demand
# model_name = "HooshvareLab/bert-fa-base-uncased-sentiment-snappfood" 
# model_name = "HooshvareLab/bert-fa-base-uncased-sentiment-digikala" 
# model_name = "HooshvareLab/bert-fa-zwnj-base" 

texts = [
    "من این محصول رو دوست داشتم",  # noqa
    "حالم از این وضع بهم می‌خوره",  # noqa
]

for text in texts:
    label, probability = predict_parsbert(model_name, cache_dir, device, label_dict, text, base_path)
    print(f"Text: {text}")
    print(f"Predicted label: {label} with probability: {probability}")
    print('*' * 50)

Text: من این محصول رو دوست داشتم
Predicted label: HAPPY with probability: 0.5366262197494507
**************************************************
Text: حالم از این وضع بهم می‌خوره
Predicted label: SAD with probability: 0.504980742931366
**************************************************
